In [58]:
import math
import matplotlib.pyplot as plt
import keras
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
df=pd.read_csv("/content/BERGERPBL.csv",parse_dates=['DATE'])
df.head()

#       DATE TRADING CODE    LTP*  ...     YCP   TRADE  VALUE (mn)   VOLUME
0  439 2018-05-08    BERGERPBL  1265.0  ...  1275.4   256.0        4.60   3610.0
1  438 2018-06-08    BERGERPBL  1285.0  ...  1265.3   222.0        6.02   4735.0
2  437 2018-07-08    BERGERPBL  1244.0  ...  1277.5  1958.0       84.90  68318.0
3  436 2018-08-08    BERGERPBL  1215.0  ...  1235.2  1729.0       49.76  41094.0
4  435 2018-09-08    BERGERPBL  1194.7  ...  1210.4  1276.0       24.90  20785.0

[5 rows x 12 columns]

In [59]:
# Sort DataFrame by date
df = df.sort_values('DATE')

# Double check the result
df.head()

#       DATE TRADING CODE    LTP*  ...     YCP   TRADE  VALUE (mn)   VOLUME
36  403 2018-01-10    BERGERPBL  1250.4  ...  1250.6   552.0       11.42   9109.0
59  380 2018-01-11    BERGERPBL  1222.1  ...  1228.4   324.0        4.08   3331.0
37  402 2018-02-10    BERGERPBL  1246.1  ...  1252.2   422.0        6.19   4950.0
79  360 2018-02-12    BERGERPBL  1355.0  ...  1387.7  1472.0       44.18  31926.0
16  423 2018-03-09    BERGERPBL  1253.0  ...  1268.2   781.0       21.22  16700.0

[5 rows x 12 columns]

In [60]:
df = df.set_index('DATE')
df.head(3)

# TRADING CODE    LTP*    HIGH  ...     YCP  TRADE  VALUE (mn)  VOLUME
DATE                                          ...                                   
2018-01-10  403    BERGERPBL  1250.4  1278.9  ...  1250.6  552.0       11.42  9109.0
2018-01-11  380    BERGERPBL  1222.1  1234.0  ...  1228.4  324.0        4.08  3331.0
2018-02-10  402    BERGERPBL  1246.1  1268.0  ...  1252.2  422.0        6.19  4950.0

[3 rows x 11 columns]

In [61]:
df['daily_ret'] = df['CLOSEP*'].pct_change()
df['daily_ret'] = df['daily_ret'].fillna(0.0)

In [62]:
df['volatility']= df['daily_ret'].rolling(window=21).std()
df['volatility'] = df['volatility'].fillna(0.0)

In [63]:
df['volatility'] = df['volatility']*100


In [64]:
import matplotlib.pyplot as plt 

In [65]:
df=df.reset_index()['volatility']

In [66]:
df.shape

(439,)

In [67]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler(feature_range=(0,1))
df=scaler.fit_transform(np.array(df).reshape(-1,1))

In [68]:
##splitting dataset into train and test split where train=70 percent and test=30 percent
training_size=int(len(df)*0.70)
test_size=len(df)-training_size
train_data,test_data=df[0:training_size,:],df[training_size:len(df),:1]




In [69]:
import numpy
# convert an array of values into a dataset matrix
def create_dataset(dataset, time_step=1):
	dataX, dataY = [],[]
	for i in range(len(dataset)-time_step-1):
		a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100 
		dataX.append(a)
		dataY.append(dataset[i + time_step, 0])
	return numpy.array(dataX), numpy.array(dataY)

In [70]:

# reshape into X=t,t+1,t+2,t+3 and Y=t+4
time_step = 100
X_train, y_train = create_dataset(train_data, time_step)
X_test, ytest = create_dataset(test_data, time_step)

In [71]:
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor,GradientBoostingRegressor
import xgboost as xgb
from sklearn.datasets import make_regression



In [72]:
model1 = RandomForestRegressor(n_estimators=1000)
model2 =AdaBoostRegressor(n_estimators=1000,learning_rate=0.01)
model3= GradientBoostingRegressor(n_estimators=1000,learning_rate=0.01)
model4 = xgb.XGBRegressor(n_estimators=1000,learning_rate=0.01)



In [73]:
model1.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=1000, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [74]:
model2.fit(X_train, y_train)

AdaBoostRegressor(base_estimator=None, learning_rate=0.01, loss='linear',
                  n_estimators=1000, random_state=None)

In [75]:
model3.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.01, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=1000,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [76]:
model4.fit(X_train, y_train)

[14:30:45] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.01, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=1000,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [77]:

train_predict1=model1.predict(X_train)
test_predict1= model1.predict(X_test)

In [78]:

train_predict2=model2.predict(X_train)
test_predict2= model2.predict(X_test)

In [79]:

train_predict3=model3.predict(X_train)
test_predict3= model3.predict(X_test)

In [80]:

train_predict4=model4.predict(X_train)
test_predict4= model4.predict(X_test)

In [81]:
Y_test = np.reshape(ytest,(1, ytest.size))

In [82]:
Y_test =scaler.inverse_transform(Y_test)

In [83]:
Y_test

array([[2.51368837, 2.11558746, 2.07551226, 1.38372562, 1.38372562,
        1.38372562, 1.38372562, 1.38372562, 1.38372562, 1.39126503,
        1.39986198, 1.39986198, 1.39986198, 1.50335943, 1.57719186,
        1.90821559, 1.90821559, 1.61726801, 2.15445503, 2.47684157,
        2.49513074, 2.51708613, 2.79752762, 3.00037658, 3.05021567,
        3.4470819 , 3.56497017, 3.64432578, 3.65854508, 3.86230258,
        3.99987419]])

In [84]:
test_predict1 = np.reshape(test_predict1,(1, test_predict1.size))
test_predict2 = np.reshape(test_predict2,(1, test_predict2.size))
test_predict3 = np.reshape(test_predict3,(1, test_predict3.size))
test_predict4 = np.reshape(test_predict4,(1, test_predict3.size))



In [85]:
test_predict1 =scaler.inverse_transform(test_predict1)
test_predict2 =scaler.inverse_transform(test_predict2)
test_predict3 =scaler.inverse_transform(test_predict3)
test_predict4 =scaler.inverse_transform(test_predict4)

In [86]:
test_predict1

array([[7.52694368, 7.52929097, 7.53262188, 7.53007778, 7.533306  ,
        7.5308965 , 7.53577151, 7.53064923, 7.53223632, 7.53054571,
        7.5311693 , 7.53151272, 7.5307413 , 7.53203766, 7.5304107 ,
        7.5287745 , 7.52664482, 7.53074868, 7.5333631 , 7.53567775,
        7.53998594, 7.54215055, 7.54508564, 7.54619746, 7.54836539,
        7.55394877, 7.55650964, 7.5451835 , 7.51801054, 7.49886864,
        7.4999635 ]])

In [87]:
test_predict2

array([[ 9.97886634,  9.9747265 ,  9.97886634,  9.97900526,  9.97900526,
         9.97900526,  9.97900526,  9.98105182, 10.0094759 , 10.02641136,
        10.03501775, 10.043715  , 10.05417338, 10.05079326, 10.05079326,
        10.05079326, 10.05079326, 10.05079326, 10.05079326, 10.05079326,
        10.05079326, 10.05079326, 10.05079326, 10.05079326, 10.05079326,
        10.05079326, 10.04541332, 10.05079326, 10.043715  , 10.02868762,
        10.02724567]])

In [88]:
test_predict3

array([[8.81143669, 8.80963867, 8.91269488, 8.91269488, 8.96333563,
        8.93184785, 8.93184785, 8.93382927, 8.93382927, 8.92620242,
        8.93143474, 8.93143474, 8.98123042, 8.98469865, 8.98267377,
        8.98267377, 8.98178364, 8.92964513, 8.85635514, 8.85635514,
        8.8558609 , 8.86058037, 9.0354396 , 9.03234388, 8.73507286,
        8.71671679, 8.71671679, 8.71671679, 8.71671679, 8.72928047,
        8.72686793]])

In [89]:
test_predict4

array([[9.014486, 9.014486, 9.229316, 9.229316, 9.229316, 9.155995,
        9.155995, 9.150942, 9.269467, 9.23876 , 9.249376, 9.249376,
        9.22812 , 9.249376, 9.249376, 9.249376, 9.250338, 9.278224,
        9.274266, 9.270632, 9.270632, 9.279253, 9.279253, 9.263516,
        8.986758, 8.986758, 8.91775 , 8.91775 , 9.065848, 9.10032 ,
        9.087844]], dtype=float32)

In [90]:
#Y_test = Y_test.transpose()
RMSE1 = math.sqrt(mean_squared_error(Y_test,test_predict1))
RMSE1

5.35353593096097

In [91]:
#Y_test = Y_test.transpose()
RMSE2 = math.sqrt(mean_squared_error(Y_test,test_predict2))
RMSE2

7.824246345338759

In [92]:
#Y_test = Y_test.transpose()
RMSE3 = math.sqrt(mean_squared_error(Y_test,test_predict3))
RMSE3

6.698484870078957

In [93]:
#Y_test = Y_test.transpose()
RMSE4 = math.sqrt(mean_squared_error(Y_test,test_predict4))
RMSE4

6.987530970130876

In [94]:

from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(Y_test,test_predict1)
print('MAE: %f' % mae)

MAE: 5.281829


In [95]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(Y_test,test_predict2)
print('MAE: %f' % mae)

MAE: 7.776565


In [96]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(Y_test,test_predict3)
print('MAE: %f' % mae)

MAE: 6.630548


In [97]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(Y_test,test_predict4)
print('MAE: %f' % mae)

MAE: 6.922943
